<p align="center">
  <img src="img\logo.png" alt="laliga">
</p>

# Exploratory data analysis: LaLiga  

#### Analisis sobre el rendimiento de los equipos en distintas condiciones

##### Índice:

- Introducción y motivación.

- Obtencion de Datos.

- Nomenclatura de los Datos.

- Herramientas para la utilización de datos.

- Uso de notebooks.

- Hipótesis: 
    - Los equipos con mejor rendimiento en LaLiga mantienen rachas de victorias y son capaces de evitar las de derrotas
    - Los equipos con mejor desempeño en LaLiga tienen una mayor cantidad de disparos. Además son más eficaces que los demás.
    - Los equipos tienen un mejor rendimiento cuando juegan de locales: marcan más y les marcan menos.
    - Existe un factor de horario que influye a algunos partidos.
    - ¿Existe un número de goles que tenga mayor frecuencia que otros para algún equipo en específico?
    - Los equipos con mayor presupuesto son los que mejor eligen los fichajes mientras que los de la tabla más baja recurren a fichajes más baratos. Se busca un factor de influencia en la nacionalidad de los jugadores extranjeros.
    - ¿La edad de la plantilla inlfuye?
    

#### Introducción y motivación:

En este trabajo de EDA (Exploratory data analysis), trataremos de analizar a los distintos equipos de LaLiga en la temporada 22/23 de forma que podamos sacar conclusiones mediante estadística descriptiva en cuanto a su desempeño en los partidos. 

La necesidad de hacer este trabajo viene dada por la busqueda de patrones de eventos como pueden ser rachas de goles o mejoras en el rendimiento a la hora de jugar como equipo local. 

#### Obtención de datos:

Se han tomado dos tipos de datos en este trabajo. El primero, en formato '.csv', tomado de la página [football-data](https://www.football-data.co.uk/), contiene los datos de cada uno de los partidos que se jugaron en la temporada 22/23 de LaLiga. El segundo, obtenido mediante webscrapping, se han tomado de la página de futbol [transfermarkt](https://www.transfermarkt.es/), que contiene datos de cada una de las plantillas que tuvieron los equipos en dicha temporada.

#### Nomenclatura de los datos de los partidos:

El archivo 'SP1.csv' obtenido que contiene los datos de los partidos de la temporada contiene muchas columnas que no nos hacen falta, por ello, se han tomado solamente las columnas que tienen sentido considerarse en un EDA como este. Son las siguientes:

- Date: Contiene la fecha en la que se jugó el partido.
- Time: La hora a la que se jugó el partido.
- HomeTeam: El equipo que jugaba de local
- AwayTeam: El equipo que jugaba de visitante.
- FTHG: Goles totales marcados por el equipo local.
- FTAG: Goles totales marcados por el equipo visitante.
- FTR: Resultado final del partido. 'H' si ganó el equipo local, 'D' si el resultado fué empate, 'A' si ganó el equipo visitante.
- HTHG: Goles marcados por el equipo local en la primera mitad del partido.
- HTAG: Goles marcados por el equipo visitante en la primera mitad del partido.
- HTR: Resultado a mitad del partido. 'H' si ganó el equipo local, 'D' si el resultado fué empate, 'A' si ganó el equipo visitante.
- HS: Tiros a puerta totales del equipo local.
- AS: Tiros a puerta totales del equipo visitante.
- HST: Tiros totales del equipo local.
- AST: Tiros totales del equipo visitante.
- HF: Faltas realizadas por el equipo local.
- AF: Faltas realizadas por el equipo visitante.
- HC: Corners realizados por el equipo local.
- AC: Corners realizados por el equipo visitante.
- HY: Tarjetas amarillas del equipo local.
- AY: Tarjetas amarillas del equipo visitante.
- HR: Tarjetas rojas del equipo local.
- AR: Tarjetas rojas del equipo visitante.


#### Herramientas:

##### Las librerias utilizadas:

En este proyecto se han utilizado las siguientes librerias: BeautifulSoup, fake_useragent, matplotlib.pyplot, numpy pandas, requests y seaborn. Se han importado de la siguiente forma:

In [ ]:
from bs4 import BeautifulSoup 
from fake_useragent import UserAgent  
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns


Para facilitar la comprensión del código, se ha utilizado un Jupyter Notebook que contiene distintas funciones que simplifican el trabajo a realizar. Este notebook es [herlpers](helpers.py). A continuación explicaremos cada una de las funciones que contiene.

Para entender mejor la necesidad de crear estas funciones, veamos como es la tabla que contiene los datos de los jugadores del [Athletic Club](https://www.transfermarkt.es/athletic-club/kader/verein/621/plus/0/galerie/0?saison_id=2022):

<p align="center">
  <img src="img\tabla_ejemplo.png" alt="Ejemplo tabla">
</p>

In [ ]:
def get_soup(url):
    '''
    Esta función crea la 'Soup' mediante BeautifulSoup
    '''
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    response = requests.get(url, headers=headers)
    soup=BeautifulSoup(response.text, 'html.parser')
    return soup

In [ ]:
def nombres(soup):
    '''
    Esta función devuelve una lista con los nombres de los jugadores
    '''
    list=[]
    jugadores = soup.find("table", class_="items").find_all("table", class_="inline-table")
    for jug in jugadores:
        link=jug.find("a")
        nombre=link.text.strip()
        list.append(nombre)
    return list

In [ ]:
def posiciones(soup):
    '''
    Esta función devuelve una lista con las posiciones de los jugadores
    '''
    list=[]
    jugadores = soup.find("table", class_="items").find_all("td", class_="posrela")
    for jg in jugadores:
        posi=jg.find_all("tr")[-1]
        list.append(posi.text.strip())
    return list

In [ ]:
def edades(soup):
    '''
    Esta función devuelve una lista con las edades de los jugadores
    '''
    list=[]
    jugadores = soup.find("table", class_="items").find_all("td",class_="zentriert")
    for i in range(len(jugadores)):
        if i%4==1:
            edad=jugadores[i]   
            list.append(edad.text.strip())
    return list

In [ ]:
def nacionalidades(soup):
    '''
    Esta función devuelve una lista con las nacionalidades de los jugadores
    '''
    list=[]
    jugadores = soup.find("table", class_="items").find_all("td",class_="zentriert")
    for i in range(len(jugadores)):
        if i%4==2:
            nac=jugadores[i].find("img")['title']
            list.append(nac)
    return list

In [ ]:
def valores(soup):
    '''
    Esta función devuelve una lista con los valores de mercado de los jugadores
    '''
    list=[]
    jugadores = soup.find("table", class_="items").find_all("td", class_="rechts hauptlink")
    for jg in jugadores:
        precio=jg.find("a")
        if precio:
            list.append(precio.text.strip())
        else:
            list.append("Desconocido")
    return list

In [ ]:
def crear_df(soup):
    '''Crea un DataFrame de un equipo de futbol a partir de una sopa de BeautifulSoup con Nombre, Edad, Posición
     Nacionalidad y Valor de Mercado '''
    dict = {'Nombre' : nombres(soup),
        'Edad':edades(soup),
        'Posición' : posiciones(soup),
        'Nacionalidad': nacionalidades(soup),
        'Valor_Mercado':valores(soup)
    }
    df=pd.DataFrame(dict)
    return df

Esta última función nos devuelve un dataframe que contiene los datos de una tabla similar a la que se ha mostrado en la imagen inicial. En el caso del Athletic club, las primeras filas del DataFrame que devuelve son:

<p align="center">
  <img src="img\df_ejemplo.png" alt="Ejemplo df">
</p>

Cada uno de los distintos pasos para obtener los archivos '.csv' se ha realizado en [CrearData](CrearData.ipynb) mediante el siguiente código:

In [ ]:
for key,value in links.items(): #Donde link es un diccionario que contiene como key el nombre del equipo y value el link a la página
    df=crear_df(get_soup(value))
    df.to_csv('./Data/'+key+'.csv', index=False)

Ahora, en ese mismo archivo, se hace la limpieza de los datos del archivo que contiene los datos de los partidos: [SP1](/Data/SP1.csv) ya que los nombres de los equipos no coinciden con los nombres de los equipos que hemos utilizado para crear los archivos '.csv' de las platillas. Además, se eliminan las columnas que no son de interes y guardamos el nuevo DataFrame en el archivo [0.Partidos](/Data/0.Partidos.csv).

El siguiente paso es crear un último archivo '.csv' que contenga la tabla de clasificación de LaLiga con algunos datos de interes. Para ello, se han utilizado las siguientes funciones de [herlpers](helpers.py):

In [ ]:
def data_equipo(df,equipo):
    '''Dado un data frame con datos de partidos de futbol, devuelve un dataframe especifico de
    los datos de un equipo concreto'''
    partidos_equipo=df[(df['HomeTeam'] == equipo) | (df['AwayTeam'] == equipo)]
    return partidos_equipo

In [ ]:
def victorias(df_equipo,equipo):
     '''Dado un data frame con los datos de los partidos de un equipo, devuelve el número de victorias de ese equipo'''
     return len(df_equipo[(df_equipo['HomeTeam'] == equipo) & (df_equipo['FTR'] == 'H')]) + len(df_equipo[(df_equipo['AwayTeam'] == equipo) & (df_equipo['FTR'] == 'A')])

In [ ]:
def empates(df_equipo,equipo):
    '''Dado un data frame con los datos de los partidos de un equipo, devuelve el número de empates de ese equipo'''
    return len(df_equipo[(df_equipo['HomeTeam'] == equipo) & (df_equipo['FTR'] == 'D')]) + len(df_equipo[(df_equipo['AwayTeam'] == equipo) & (df_equipo['FTR'] == 'D')])

In [ ]:
def derrotas(df_equipo,equipo):
    '''Dado un data frame con los datos de los partidos de un equipo, devuelve el número de derrotas de ese equipo'''
    return len(df_equipo[(df_equipo['HomeTeam'] == equipo) & (df_equipo['FTR'] == 'A')]) + len(df_equipo[(df_equipo['AwayTeam'] == equipo) & (df_equipo['FTR'] == 'H')])

In [ ]:
def goles_marcados(df_equipo,equipo):
    '''Devuelve el número de goles que ha marcado el equipo en la temporada'''
    return df_equipo[df_equipo['HomeTeam'] == equipo]['FTHG'].sum() + df_equipo[df_equipo['AwayTeam'] == equipo]['FTAG'].sum()

In [ ]:
def goles_en_contra(df_equipo,equipo):
    '''Devuelve el número de goles que le han marcado al equipo en la temporada'''
    return df_equipo[df_equipo['HomeTeam'] == equipo]['FTAG'].sum() + df_equipo[df_equipo['AwayTeam'] == equipo]['FTHG'].sum()
    

Las siguientes dos funciones se utilizan más adelante:

In [ ]:
def data_local(df_equipo,equipo):
    '''Devuelve un DataFrame con los partidos que ha jugado de Local un equipo '''
    df_local=df_equipo[df_equipo['HomeTeam']==equipo]
    return df_local

In [ ]:
def data_visitante(df_equipo,equipo):
    '''Devuelve un DataFrame con los partidos que ha jugado de Visitante un equipo '''
    df_visitante=df_equipo[df_equipo['AwayTeam']==equipo]
    return  df_visitante

Con estas nuevas funciones, de nuevo en el archivo [CrearData](CrearData.ipynb) se crea la tabla en un DataFrame:

In [ ]:
# Está ordenada en función de la posición final de cada equipo
lista_equipos=['FC Barcelona',
 'Real Madrid',
 'Atlético de Madrid',
 'Real Sociedad',
 'Villarreal CF',
 'Real Betis',
 'Osasuna',
 'Athletic Club',
 'RCD Mallorca',
 'Girona FC',
 'Sevilla FC',
 'Rayo Vallecano',
 'Celta de Vigo',
 'Valencia CF',
 'Getafe CF',
 'Cadiz CF',
 'UD Almería',
 'Real Valladolid',
 'RCD Espanyol',
 'Elche CF']

In [ ]:
d={'Equipo': lista_equipos,
    'V' : [victorias(data_equipo(df_partidos,equipo),equipo) for equipo in lista_equipos],
    'E' : [empates(data_equipo(df_partidos,equipo),equipo) for equipo in lista_equipos],
    'D' : [derrotas(data_equipo(df_partidos,equipo),equipo) for equipo in lista_equipos],
    'Goles a favor': [goles_marcados(data_equipo(df_partidos,equipo),equipo) for equipo in lista_equipos],
    'Goles en contra': [goles_en_contra(data_equipo(df_partidos,equipo),equipo) for equipo in lista_equipos]
}
df_clasi=pd.DataFrame(d)

In [ ]:
# Añadimos los puntos de los equipos:
puntos=3*df_clasi['V'] + df_clasi['E']
df_clasi['Puntos']=puntos
df_clasi


Finalmente guardamos la tabla en [1.Clasificacion](/Data/1.Clasificacion.csv).

In [ ]:
df_clasi.to_csv('./Data/1.Clasificacion.csv', index=False)

La tabla resultante es:

<p align="center">
  <img src="img\tabla_clasificacion.png" alt="Tabla Clasificacion">
</p>

#### Resultados de los equipos:

En [nb_clasificacion](nb_clasificacion.ipynb) se han realizado distintas técnicas para la obtención de datos acerca de la clasificación final de los jugadores. En la primera parte del Notebook, se calculan las series temporales con los resultados de cada uno de los equipos donde se ha ponderado una victoria como +1, 0 como empate y -1 para las derrotas. Estos datos se encuentran en la carpeta *Serie Temporal*. Los resultados son los siguientes en orden de la clasificación final:

Para los 4 lideres de la tabla tenemos:

<p align="center">
  <img src="Serie Temporal\FC Barcelona.png" alt="Tabla Clasificacion">
  <img src="Serie Temporal\Real Madrid.png" alt="Tabla Clasificacion">
  <img src="Serie Temporal\Atlético de Madrid.png" alt="Tabla Clasificacion">
  <img src="Serie Temporal\Real Sociedad.png" alt="Tabla Clasificacion">
</p>

Para los 4 últimos de la tabla tenemos:

<p align="center">
  <img src="Serie Temporal\UD Almería.png" alt="Tabla Clasificacion">
  <img src="Serie Temporal\Real Valladolid.png" alt="Tabla Clasificacion">
  <img src="Serie Temporal\RCD Espanyol.png" alt="Tabla Clasificacion">
  <img src="Serie Temporal\Elche CF.png" alt="Tabla Clasificacion">
</p>

Podemos comprobar que los lideres mantinen la racha de victorias de forma regular mientras que los ultimos de la liga son más inconsistentes y no suelen tener más de dos victorias consecutivas.

Si analizamos a los equipos de media tabla tenemos lo siguiente:

<p align="center">
  <img src="Serie Temporal\RCD Mallorca.png" alt="Tabla Clasificacion">
  <img src="Serie Temporal\Girona FC.png" alt="Tabla Clasificacion">
  <img src="Serie Temporal\Sevilla FC.png" alt="Tabla Clasificacion">
</p>

Estos equipos tienen mayor frecuencia de victorias que los de la tabla baja (como era de esperar) pero podemos seguir viendo que no existe una evidencia de que una vez un equipo ha conseguido una victoria, la mantenga para el siguiente partido. Sin embargo, podemos observar que estos equipos, a diferencia de los de la tabla baja, tienen menos empates y oscilan entre victorias y derrotas.

A continuación, en ese mismo notebook, se obtienen los datos relativos a cada equipo acerca de los tiros, tiros a puerta, goles (de local y visitante) y distintos ratios entre tiros por gol de cada equipo. Para ello, se utiliza de nuevo [0.Partidos](/Data/0.Partidos.csv).

Se ha utilizado el siguiente código:

In [ ]:
d_goles_tiros=dict() #diccionario que guardará como key los equipos y como value el numero de partidos en los que han metido un gol por cada 2 tiros a puerta
d_goles_tiros_total=dict() #diccionario que guardará como key los equipos y como value el numero de partidos en los que han metido un gol por cada 4 tiros
d_tiros=dict() #diccionario que guardará como key los equipos y como value el numero de tiros 
d_tiros_a_puerta=dict() #diccionario que guardará como key los equipos y como value el numero de tiros a puerta
d_goles=dict() #diccionario que guardará como key los equipos y como value el numero de goles 
for equipo in df_partidos['HomeTeam'].unique():
    df_equipo=data_equipo(df_partidos,equipo) #Creamos el dataframe del equipo
    df_local=data_local(df_equipo,equipo)           #Creamos el dataframe del equipo cuando juega de local
    df_visitante=data_visitante(df_equipo,equipo)   #Creamos el dataframe del equipo cuando juega de visitante
    df_equipo=df_equipo.reset_index()
    df_local=df_local.reset_index()
    df_visitante=df_visitante.reset_index()
    #creamos una nueva columna con los tiros a puerta, tiros totales del equipo en cada partido y goles que ha metido el equipo en cada partido.
    df_equipo['tiros_a_puerta_equipo']=np.array(range(len(df_equipo)))
    df_equipo['tiros_equipo']=np.array(range(len(df_equipo)))
    df_equipo['goles_equipo']=np.array(range(len(df_equipo)))
    for i, date in enumerate(df_equipo['Date']):
            if date in df_local['Date'].values:
                k = np.where(df_local['Date'].values == date)[0][0]
                df_equipo.at[i, 'tiros_a_puerta_equipo'] = df_local.at[k, 'HST']
                df_equipo.at[i, 'tiros_equipo'] = df_local.at[k, 'HS']
                df_equipo.at[i, 'goles_equipo'] = df_local.at[k, 'FTHG']
            else:
                k = np.where(df_visitante['Date'].values == date)[0][0]
                df_equipo.at[i, 'tiros_a_puerta_equipo'] = df_visitante.at[k, 'AST']
                df_equipo.at[i, 'tiros_equipo'] = df_visitante.at[k, 'AS']  
                df_equipo.at[i, 'goles_equipo'] = df_visitante.at[k, 'FTAG']
    d_goles[equipo]=sum(df_equipo['goles_equipo'])
    d_tiros[equipo]=sum(df_equipo['tiros_equipo'])
    d_tiros_a_puerta[equipo]=sum(df_equipo['tiros_a_puerta_equipo'])
    ratio_tiros=df_equipo['tiros_a_puerta_equipo']/df_equipo['tiros_equipo']
    df_equipo['ratio_tiros']=ratio_tiros
    goles_por_tiro=df_equipo['goles_equipo']/df_equipo['tiros_equipo']
    df_equipo['goles_por_tiro']=goles_por_tiro
    goles_por_tiro_a_puerta=df_equipo['goles_equipo']/df_equipo['tiros_a_puerta_equipo']
    df_equipo['goles_por_tiro_a_puerta']=goles_por_tiro_a_puerta
    d_goles_tiros[equipo]=len(df_equipo[df_equipo['goles_por_tiro_a_puerta']>=0.5])
    d_goles_tiros_total[equipo]=len(df_equipo[df_equipo['goles_por_tiro']>=0.25])

In [ ]:
#Creamos unas lista de tuplas con los diccionarios obtenidos
l_goles_tiros=list(d_goles_tiros.items())
l_goles_tiros_total=list(d_goles_tiros_total.items())
l_tiros=list(d_tiros.items())
l_tiros_a_puerta=list(d_tiros_a_puerta.items())
l_goles=list(d_goles.items())
#Los ordenamos de forma que sigan el mismo orden que la clasificación de la liga
l_goles_tiros = sorted(l_goles_tiros, key=lambda x: list(df_clasi['Equipo']).index(x[0]))
l_goles_tiros_total = sorted(l_goles_tiros_total, key=lambda x: list(df_clasi['Equipo']).index(x[0]))
l_tiros = sorted(l_tiros, key=lambda x: list(df_clasi['Equipo']).index(x[0]))
l_tiros_a_puerta = sorted(l_tiros_a_puerta, key=lambda x: list(df_clasi['Equipo']).index(x[0]))
l_goles = sorted(l_goles, key=lambda x: list(df_clasi['Equipo']).index(x[0]))

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=df_clasi['Equipo'], y=[tp[1] for tp in l_goles_tiros], color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Total', fontsize=14)
plt.title('Número de veces que un equipo mete un gol por cada 2 tiros a puerta realizados', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in [tp[1] for tp in l_goles_tiros]:
    plt.text(ubicaciones[c], i + 0.2, str(i), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Goles por tiros a puerta.png" alt="Goles/Tiros a puerta">
</p>

Podemos ver en general todos los equipos tiene una buena calificación en este apartado, sin embargo, veremos como esto a medida que vamos acotando los datos va a ser más dispar.

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=df_clasi['Equipo'], y=[tp[1] for tp in l_goles_tiros_total], color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Total', fontsize=14)
plt.title('Número de veces que un equipo mete un gol por cada 4 tiros realizados', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in [tp[1] for tp in l_goles_tiros_total]:
    plt.text(ubicaciones[c], i + 0.05, str(i), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Goles por tiros.png" alt="Goles/Tiros">
</p>

En este caso, podemos comprobar como los de la tabla alta tiene una mejor calidad de tiro respecto a los demás ya que tienen hasta el doble de goles por cada 4 tiros realizados que los de la tabla baja. Podemos concluir que efectivamente, tienen una mayor efectividad de tiro/gol.

Además, ahora comprobaremos que los equipos con mejor ratio de tiro/gol tienen además una mayor cantidad de tiros realizados y por lo tanto mayor numero de goles:

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=df_clasi['Equipo'], y=[tp[1] for tp in l_tiros], color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Total', fontsize=14)
plt.title('Tiros totales realizados por cada equipo', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in [tp[1] for tp in l_tiros]:
    plt.text(ubicaciones[c], i + 0.2, str(i), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Tiros.png" alt="Tiros">
</p>

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=df_clasi['Equipo'], y=[tp[1] for tp in l_tiros_a_puerta], color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Total', fontsize=14)
plt.title('Tiros a puerta totales realizados por cada equipo', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in [tp[1] for tp in l_tiros_a_puerta]:
    plt.text(ubicaciones[c], i + 0.2, str(i), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Tiros a puerta.png" alt="Tiros a puerta">
</p>

Por lo tanto, como era de esperar, los equipos de la tabla alta tienen una mayor cantidad de goles:

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=df_clasi['Equipo'], y=[tp[1] for tp in l_goles], color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Total', fontsize=14)
plt.title('Goles por cada equipo', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in [tp[1] for tp in l_goles]:
    plt.text(ubicaciones[c], i + 0.2, str(i), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Goles.png" alt="Goles">
</p>

Y como era de esperar, los equipos que estan por debajo de la tabla reciben más goles:

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=df_clasi['Equipo'], y=df_clasi['Goles en contra'], color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Total', fontsize=14)
plt.title('Goles en contra por cada equipo', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in df_clasi['Goles en contra']:
    plt.text(ubicaciones[c], i + 0.2, str(i), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\goles en contra.png" alt="Goles">
</p>

Sin embargo, existe un factor que hace que un equipo tenga mayor oportunidad de marcar en un partido concreto: si juega de local o no.

Podemos comprobarlo en los siguientes diagramas:

In [ ]:
#Goles como equipo local y como visitante
goles_local=[]
goles_visitante=[]
for equipo in df_clasi['Equipo']:
    df_equipo=data_equipo(df_partidos,equipo)
    df_local=data_local(df_equipo,equipo)
    df_visitante=data_visitante(df_equipo,equipo)
    goles_local.append(sum(df_local['FTHG']))
    goles_visitante.append(sum(df_visitante['FTAG']))

sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=df_clasi['Equipo'], y=goles_local, color='skyblue',label='Goles Local')
sns.barplot(x=df_clasi['Equipo'], y=goles_visitante, color='orange',label='Goles Visitante')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Total', fontsize=14)
plt.title('Goles a favor', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
plt.show()

<p align="center">
  <img src="Diagramas de barras\Goles separados.png" alt="Goles">
</p>

Además, este mismo factor también influye en los goles que recibe el equipo en un mismo partido:

In [ ]:
#Goles como equipo local y como visitante
goles_contra_local=[]
goles_contra_visitante=[]
for equipo in df_clasi['Equipo']:
    df_equipo=data_equipo(df_partidos,equipo)
    df_local=data_local(df_equipo,equipo)
    df_visitante=data_visitante(df_equipo,equipo)
    goles_contra_local.append(sum(df_local['FTAG']))
    goles_contra_visitante.append(sum(df_visitante['FTHG']))

sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=df_clasi['Equipo'], y=goles_contra_visitante, color='orange',label='Goles en Contra de Visitante')
sns.barplot(x=df_clasi['Equipo'], y=goles_contra_local, color='skyblue',label='Goles en Contra de Local')

plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Total', fontsize=14)
plt.title('Goles en contra', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.legend()
plt.show()

<p align="center">
  <img src="Diagramas de barras\Goles en contra separados.png" alt="Goles">
</p>

Por lo tanto, podemos concluir que un equipo cuando juega de local es más probable que reciba menos goles y más probable que marque más y si es de visitante, al reves. Por lo que es de esperar que un equipo tenga mayor número de victorias de local que de visitante.

Para comprobarlo, debemos ir a [nb_partidos](nb_partidos.ipynb) donde hacemos la comprobación para cada uno de los equipos de la liga. Cada uno de los diagramas se encuentra en *Histograma Resultados*.

In [ ]:
#Cargamos el fichero de los partidos de 22/23
df = pd.read_csv('./Data/0.Partidos.csv')
df.head()

In [ ]:
#Obtenemos una lista con todos los nombres de los equipos
lista_equipos=df['HomeTeam'].unique()
lista_equipos

In [ ]:
for equipo in lista_equipos:
    plt.figure();
    df_equipo=data_equipo(df,equipo)
    datos_local=data_local(df_equipo,equipo)
    datos_visitante=data_visitante(df_equipo,equipo)
    sns.histplot(datos_local['FTR'],alpha=0.5, label='Local');
    sns.histplot(datos_visitante['FTR'],alpha=0.5, label='Visitante');
    plt.legend();
    plt.title('Resultados ' + equipo);
    plt.xlabel('Resultado')
    plt.ylabel('Cantidad')
    plt.savefig('./Histograma Resultados/'+ equipo+'.png');

De forma resumida, podemos comprobar que para los primeros de la clasificación pomdeos obtener los siguientes diagramas:

<p align="center">
  <img src="Histograma Resultados\FC Barcelona.png" alt="hist">
  <img src="Histograma Resultados\Real Madrid.png" alt="hist">
  <img src="Histograma Resultados\Atlético de Madrid.png" alt="hist">
  <img src="Histograma Resultados\Real Sociedad.png" alt="hist">
</p>

Para los ultimos de la tabla:

<p align="center">
  <img src="Histograma Resultados\UD Almería.png" alt="Tabla Clasificacion">
  <img src="Histograma Resultados\Real Valladolid.png" alt="Tabla Clasificacion">
  <img src="Histograma Resultados\RCD Espanyol.png" alt="Tabla Clasificacion">
  <img src="Histograma Resultados\Elche CF.png" alt="Tabla Clasificacion">
</p>

En todos los casos se comprueba que es más probable que gane que cuando un equipo juega de local que cuando juega de visitante.

¿Existe algún factor que también influya en la victoria de un equipo? Para ver si podemos responder a esta pregunta, vamos a analizar la hora a la que se jugan los partidos y comprobar si existe algún en las victorias de un equipo. Para ello, continuamos en [nb_partidos](nb_partidos.ipynb):

En esta parte, catalogaremos los partidos en función de su hora de empiece: hasta las 12:59 como 'Mañana', de 13:00 a 15:59 como 'Mediodia', de 16:00 a 19:00 como 'Tarde' y el resto como 'Noche'.

In [ ]:
df_partidos = pd.read_csv('./Data/0.Partidos.csv')
df_clasi = pd.read_csv('./Data/1.Clasificacion.csv')

In [ ]:
df_partidos['Horas Decimal']=np.array(range(len(df_partidos)))
c=0
for i in df_partidos['Time']:
    horas, minutos = i.split(':')
    df_partidos.at[c,'Horas Decimal']=int(horas) + int(minutos)/60
    c+=1
df_partidos
intervalos = [0, 13, 16, 19,24]
etiquetas = ['Mañana', 'Mediodia', 'Tarde','Noche']
df_partidos['Horario'] = pd.cut(df_partidos['Horas Decimal'], bins=intervalos, labels=etiquetas)
df_partidos

In [ ]:
l_mañana=[]
l_mañana_jugados=[]
l_mediodia=[]
l_mediodia_jugados=[]
l_tarde=[]
l_tarde_jugados=[]
l_noche=[]
l_noche_jugados=[]
for equipo in df_clasi['Equipo']:
    df_equipo=data_equipo(df_partidos,equipo)
    df_equipo_mañana=df_equipo[df_equipo['Horario']=='Mañana']
    l_mañana_jugados.append(len(df_equipo_mañana))
    if len(df_equipo_mañana)>0:
        l_mañana.append(round(victorias(df_equipo_mañana,equipo)/len(df_equipo_mañana),2))  
    else:
        l_mañana.append(0)
    df_equipo_mediodia=df_equipo[df_equipo['Horario']=='Mediodia']
    l_mediodia_jugados.append(len(df_equipo_mediodia))
    if len(df_equipo_mediodia)>0:
        l_mediodia.append(round(victorias(df_equipo_mediodia,equipo)/len(df_equipo_mediodia),2))
    else:
        l_mediodia.append(0)
    l_tarde_jugados.append(len(df_equipo_tarde))
    df_equipo_tarde=df_equipo[df_equipo['Horario']=='Tarde']
    if len(df_equipo_tarde)>0:
        l_tarde.append(round(victorias(df_equipo_tarde,equipo)/len(df_equipo_tarde),2))
    else:
        l_tarde.append(0)
    l_noche_jugados.append(len(df_equipo_noche))
    df_equipo_noche=df_equipo[df_equipo['Horario']=='Noche']
    if len(df_equipo_noche)>0:
        l_noche.append(round(victorias(df_equipo_noche,equipo)/len(df_equipo_noche),2))
    else:
        l_noche.append(0)

In [ ]:
#Diagrama de barras mañana
equipos_mañana=df_clasi['Equipo']
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=equipos_mañana, y=l_mañana, color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Frecuencia Victorias', fontsize=14)
plt.title('Frecuencia de victorias partidos a la mañana', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in l_mañana:
    plt.text(ubicaciones[c], i + 0.01, 'f: '+ str(i), ha='center', va='bottom')
    c+=1
c=0
for i,value in enumerate(l_mañana):
    plt.text(ubicaciones[c], value + 0.05, 'jg: '+ str(l_mañana_jugados[i]), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Partidos a la mañana.png" alt="partidos">
</p>

In [ ]:
#Diagrama de barras mediodia
equipos_mediodia=df_clasi['Equipo']
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=equipos_mediodia, y=l_mediodia, color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Frecuencia Victorias', fontsize=14)
plt.title('Frecuencia de victorias partidos al mediodia', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in l_mediodia:
    plt.text(ubicaciones[c], i + 0.01, 'f: '+ str(i), ha='center', va='bottom')
    c+=1
c=0
for i,value in enumerate(l_mediodia):
    plt.text(ubicaciones[c], value + 0.05, 'jg: '+ str(l_mediodia_jugados[i]), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Partidos al mediodia.png" alt="partidos">
</p>

In [ ]:
#Diagrama de barras a la tarde
equipos_tarde=df_clasi['Equipo']
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=equipos_tarde, y=l_tarde, color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Frecuencia Victorias', fontsize=14)
plt.title('Frecuencia de victorias partidos a la tarde', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in l_tarde:
    plt.text(ubicaciones[c], i + 0.01, 'f: '+ str(i), ha='center', va='bottom')
    c+=1
c=0
for i,value in enumerate(l_tarde):
    plt.text(ubicaciones[c], value + 0.05, 'jg: '+ str(l_tarde_jugados[i]), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Partidos a la tarde.png" alt="partidos">
</p>

In [ ]:
#Diagrama de barras noche
equipos_noche=df_clasi['Equipo']
sns.set(style="whitegrid")
plt.figure(figsize=(20, 6))
sns.barplot(x=equipos_noche, y=l_noche, color='skyblue')
plt.xlabel('Equipos', fontsize=14)
plt.ylabel('Frecuencia Victorias', fontsize=14)
plt.title('Frecuencia de victorias partidos a la noche', fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
ubicaciones, etiquetas = plt.xticks()
c=0
for i in l_noche:
    plt.text(ubicaciones[c], i + 0.01, 'f: '+ str(i), ha='center', va='bottom')
    c+=1
c=0
for i,value in enumerate(l_noche):
    plt.text(ubicaciones[c], value + 0.05, 'jg: '+ str(l_noche_jugados[i]), ha='center', va='bottom')
    c+=1
plt.show()

<p align="center">
  <img src="Diagramas de barras\Partidos a la noche.png" alt="partidos">
</p>

En cada uno de los diagramas aparece la frecuencia de victoria 'fr' y el número de partidos jugados 'jg', además para analizar los resultados en función de la clasificación final, los equipos estan ordenados según esta tabla.

Podemos sacar las siguientes conclusiones:
- Equipos como el Celta de Vigo y el Elche CF no han ganado ni un partido a la mañana.
- Rayo Vallecano, Valencia CF y Almeria UD no han ganado ni un partido al mediodía.
- A la tarde podemos destacar que el FC Barcelona ha tenido una tasa de victoria del 83% que es muy superior a la de los demás. 
- Finalmente, a la noche, los equipos que han quedado en la mitad de la tabla para abajo, de nuevo, han ganado como mucho uno de cada 5 partidos jugados, haciendo mención especial al  RCD Español y el Elche CF que no han ganado ni un partido jugando un total de 11 y 9 partidos respectivamente. Sin embargo, los equipos de mitad de tabla hacia arriba tienen una tasa de victorias del 50% en general.

A continuación se analizará si existe una tendencia de goles para cada equipo. Para ello, volviendo a [nb_clasificacion](nb_clasificacion.ipynb):

Mediante el siguiente código, se analizarán sieres temporales con los goles por partido de cada equipo, en verde se ha gráficado la mediana de los goles y en rojo la media.

In [ ]:
for equipo in df_partidos['HomeTeam'].unique():
    df_equipo=data_equipo(df_partidos,equipo)       #Creamos el dataframe del equipo
    df_local=data_local(df_equipo,equipo)           #Creamos el dataframe del equipo cuando juega de local
    df_visitante=data_visitante(df_equipo,equipo)   #Creamos el dataframe del equipo cuando juega de visitante
    df_equipo=df_equipo.reset_index()
    df_local=df_local.reset_index()
    df_visitante=df_visitante.reset_index()
    #creamos una nueva columna con los tiros a puerta, tiros totales del equipo en cada partido y goles que ha metido el equipo en cada partido.
    df_equipo['goles_equipo']=np.array(range(len(df_equipo)))
    df_equipo['goles_contra']=np.array(range(len(df_equipo)))
    for i, date in enumerate(df_equipo['Date']):
            if date in df_local['Date'].values:
                k = np.where(df_local['Date'].values == date)[0][0]
                df_equipo.at[i, 'goles_equipo'] = df_local.at[k, 'FTHG']
                df_equipo.at[i, 'goles_contra'] = df_local.at[k, 'FTAG']
            else:
                k = np.where(df_visitante['Date'].values == date)[0][0]
                df_equipo.at[i, 'goles_equipo'] = df_visitante.at[k, 'FTAG']
                df_equipo.at[i, 'goles_contra'] = df_local.at[k, 'FTHG']
    #mediana=np.median(df_equipo['goles_equipo'])  
    #media=np.mean(df_equipo['goles_equipo'])  
    plt.figure(figsize=(45, 6));
    sns.set_style("whitegrid");
    sns.lineplot(x='Date', y='goles_equipo', data=df_equipo, label='Goles ' + equipo);
    sns.lineplot(x='Date', y='goles_contra', data=df_equipo, label='Goles rival');
    #plt.axhline(y=media, color='r', linestyle='--', label='Media')
    #plt.axhline(y=mediana, color='g', linestyle='--', label='Mediana')
    plt.xticks(rotation=45);
    plt.xlabel('Fechas');
    plt.ylabel('Goles Marcados');
    plt.title('Serie temporal de goles del ' + equipo);
    plt.legend()
    plt.savefig('./Racha goles/'+ equipo+'.png');

Para los primeros de la tabla tenemos los siguientes gráficos:

<p align="center">
  <img src="Racha goles\FC Barcelona.png" alt="hist">
  <img src="Racha goles\Real Madrid.png" alt="hist">
  <img src="Racha goles\Atlético de Madrid.png" alt="hist">
  <img src="Racha goles\Real Sociedad.png" alt="hist">
</p>

Podemos ver como el Real Madrid es el equipo que más goles promedia por partido y en general estos equipos tienen una media mayor o igual a 1 gol. En general tiene bastante regularidad en los goles,marcando entre 1 y dos goles. Además, podemos ver que las victorias suelen ser por 1 o dos goles de diferencia, salvo casos más extremos que habría que analizar con más precisión.

Para los últimos de la tabla tenemos:

<p align="center">
  <img src="Racha goles\UD Almería.png" alt="Tabla Clasificacion">
  <img src="Racha goles\Real Valladolid.png" alt="Tabla Clasificacion">
  <img src="Racha goles\RCD Espanyol.png" alt="Tabla Clasificacion">
  <img src="Racha goles\Elche CF.png" alt="Tabla Clasificacion">
</p>

Podemos comprobar como estos equipos son más irregulares y no suelen mantener una cantidad constante de goles. Es destacable que en estos 4 casos, los partidos que han ganado, lo han hecho metiendo al menos dos goles incluso podemos destacar que en muchas de esas ocasiones son 2 goles más que el equipo rival.

Para finalizar, comprobaremos varios datos acerca de los clubes para buscar si existen casos extremos que indican mayor rendimiento de la plantilla. Estos casos serán la nacionalidad de los jugadores no Españoles y de los jugadores de la platilla de cada equipo. Para ello, iremos al archivo [nb_equipos](nb_equipos.ipynb).

Para analizar las nacionalidades de las plantillas observaremos los datos de las plantillas de los equipos que se encuentran en la carpeta *Data*.

In [ ]:
df_clasi = pd.read_csv('./Data/1.Clasificacion.csv')

In [ ]:
for equipo in df_clasi['Equipo']:
    archivo='./Data/'+ str(equipo)+'.csv'
    df_platilla=pd.read_csv(archivo)
    l=[]
    for i in df_platilla['Nacionalidad'].unique():
        n=len(df_platilla[df_platilla['Nacionalidad']==i])
        tp=(i,n)
        l.append(tp)
    l=sorted(l, key=lambda x: x[1],reverse=True)
    l=l[1:]
    plt.figure(figsize=(15,10));
    sns.barplot(x=[tp[0] for tp in l], y=[tp[1] for tp in l], color='skyblue')
    plt.title('Nacionalidades del ' + equipo);
    plt.xlabel('Nacionalidades')
    plt.xticks(rotation=45);
    plt.ylabel('Cantidad')
    ubicaciones, etiquetas = plt.xticks()
    c=0
    for i in [tp[1] for tp in l]:
        plt.text(ubicaciones[c], i + 0.0001, str(i), ha='center', va='bottom')
        c+=1
    plt.savefig('./Histograma Nacionalidades/'+ equipo+'.png');

<p align="center">
  <img src="Histograma nacionalidades/FC Barcelona.png" alt="Tabla Clasificacion">
  <img src="Histograma nacionalidades/Real Madrid.png" alt="Tabla Clasificacion">
</p>

<p align="center">
    <img src="Histograma nacionalidades/Atlético de Madrid.png" alt="Tabla Clasificacion">
  <img src="Histograma nacionalidades/Real Sociedad.png" alt="Tabla Clasificacion">
</p>

Podemos obsevar que los tres primeros, al ser los clubes con mayor presupuesto, tiene la menos 10 jugadores de Paises con buen historial deportivo como son: Francia, Argentina, Brasil, Bélgica y Alemania.

<p align="center">
  <img src="Histograma nacionalidades/\UD Almería.png" alt="Tabla Clasificacion">
  <img src="Histograma nacionalidades/\Real Valladolid.png" alt="Tabla Clasificacion">
  <img src="Histograma nacionalidades/\RCD Espanyol.png" alt="Tabla Clasificacion">
  <img src="Histograma nacionalidades/\Elche CF.png" alt="Tabla Clasificacion">
</p>

Por otro lado, podemos obsevar que los tres últimos tienen en general menos de 5 jugadores de estos paises y cuentan con mayor diversidad entre Paises menos 'Tops', quitando la inesperada aparición de 9 jugadores argentinos en Elche CF.

Concluimos, como era de esperar, que los equipos que se encuentran más arriba en la clasificación son los que tienen mayor capacidad de selección de jugadores.

Finalmente analizando la edad de los jugadores de los clubes tenemos:

In [ ]:
l_edad=[]
for equipo in df_clasi['Equipo']:
    archivo='./Data/'+ str(equipo)+'.csv'
    df_platilla=pd.read_csv(archivo)
    l_edad.append(round(df_platilla['Edad'].mean(),2))
plt.figure(figsize=(20,10));
sns.barplot(x=df_clasi['Equipo'], y=l_edad, color='skyblue')
plt.title('Edad media por equipo');
plt.xlabel('Equipos')
plt.xticks(rotation=45);
plt.ylabel('Edad media')
ubicaciones, etiquetas = plt.xticks()
c=0
for i in l_edad:
    plt.text(ubicaciones[c], i + 0.1, str(i), ha='center', va='bottom')
    c+=1
plt.savefig('./Diagramas de barras/Edad media.png');

<p align="center">
  <img src="Diagramas de barras/Edad media.png" alt="Tabla Clasificacion">
</p>

In [ ]:
l_edad_mediana=[]
for equipo in df_clasi['Equipo']:
    archivo='./Data/'+ str(equipo)+'.csv'
    df_platilla=pd.read_csv(archivo)
    l_edad_mediana.append(round(df_platilla['Edad'].median(),2))
plt.figure(figsize=(20,10));
sns.barplot(x=df_clasi['Equipo'], y=l_edad_mediana, color='skyblue')
plt.title('Mediana de la edad por equipo');
plt.xlabel('Equipos')
plt.xticks(rotation=45);
plt.ylabel('Mediana de la edad')
ubicaciones, etiquetas = plt.xticks()
c=0
for i in l_edad_mediana:
    plt.text(ubicaciones[c], i + 0.1, str(i), ha='center', va='bottom')
    c+=1
plt.savefig('./Diagramas de barras/Edad mediana.png');

<p align="center">
  <img src="Diagramas de barras/Edad mediana.png" alt="Tabla Clasificacion">
</p>

De estos diagramas podemos observar que ni la media ni la mediana de la edad de los jugadores muestran una diferencia significativa en el rendimiento de los equipos